# Initial Setup.

In [1]:
import bqplot
import datetime
import dateutil.parser
import ee
import ipywidgets
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets

# Configure the pretty printing output.
pp = pprint.PrettyPrinter(depth=4)

In [2]:
# Authenticate to the Earth Engine servers, and initialize the ee module.
ee.Initialize()

# Displaying a simple, static image.

To start, let's work with a Landsat 8 image collection which has been processed to surface reflectance values (i.e. the processing has attempted to remove the atmospheric effects). You can find information on this collection by searching for "landsat 8 collection 1 SR" in the Earth Engine [Public Data Catalog](https://explorer.earthengine.google.com/#index), or by going directly to the [USGS Landsat 8 Surface Reflectance](https://explorer.earthengine.google.com/#detail/LANDSAT%2FLC08%2FC01%2FT1_SR) data description page.

The Data Description page contains the **Image ID** value *LANDSAT/LC08/C01/T1_SR*, a key piece of information that we will use in our Python code to refer to the asset.

In [3]:
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

Let's count how many images are in this collection.

In [4]:
print(l8sr.size().getInfo())

649858


To display a single image, we can filter the collection down to a single image, and can request a "thumbnail"  URL for the scene.

## Get the image's band names and store them in a variable.

In [5]:
sample_image = ee.Image(
    l8sr.filterDate('2017-11-01', '2017-12-02')
        .filterBounds(ee.Geometry.Point(-73.9957, 40.7262))
        .first()
)
band_names_original = sample_image.bandNames()
band_names_original.getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

## Rename the image bands, so they are easier to read.

In [6]:
l8_bands = ee.Dictionary({
    'B1': 'ultra_blue',
    'B2': 'blue',
    'B3': 'green',
    'B4': 'red',
    'B5': 'nir',
    'B6': 'swir_1',
    'B7': 'swir_2',
    'B8': 'pan',
    'B9': 'cirrus',
    'B10': 'tirs_1',
    'B11': 'tirs_2',
    'sr_aerosol': 'sr_aerosol', 
    'pixel_qa': 'pixel_qa',
    'radsat_qa': 'radsat_qa'
})
band_names_new = l8_bands.values(sample_image.bandNames())
l8sr = l8sr.select(band_names_original, band_names_new)

## Print out a sample image's metadata.

In [7]:
sample_image = ee.Image(
    l8sr.filterDate('2017-11-10', '2017-12-01')
        .filterBounds(ee.Geometry.Point(-73.9957, 40.7262))
        .first()
)
pp.pprint(sample_image.getInfo())

{'bands': [{'crs': 'EPSG:32618',
            'crs_transform': [30.0, 0.0, 525885.0, 0.0, -30.0, 4582215.0],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [7651, 7781],
            'id': 'ultra_blue'},
           {'crs': 'EPSG:32618',
            'crs_transform': [30.0, 0.0, 525885.0, 0.0, -30.0, 4582215.0],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [7651, 7781],
            'id': 'blue'},
           {'crs': 'EPSG:32618',
            'crs_transform': [30.0, 0.0, 525885.0, 0.0, -30.0, 4582215.0],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
           

We can request a "thumbnail" URL for the scene.

In [8]:
thumbnail_url = sample_image.getThumbUrl({
    'bands': 'red,green,blue',
    'min': 0,
    'max': 3000,
    'region': sample_image.geometry().bounds().getInfo()
})
IPython.display.HTML('Thumbnail URL: <a href={0}>{0}</a>'.format(thumbnail_url))

The image can be displayed within the notebook, using the *IPython.display.Image()* method.

In [9]:
IPython.display.Image(url=thumbnail_url)

A nice image, but not very interactive.

# Interactive Mapping

The ipyleaflet package can be used to display interactive maps. Here is a basic example:

In [10]:
import ipyleaflet
map1 = ipyleaflet.Map(zoom=3, layout={'height':'400px'})
dc = ipyleaflet.DrawControl()
map1.add_control(dc)
map1

Map(center=[0.0, 0.0], controls=(DrawControl(layer=FeatureGroup(), polygon={'shapeOptions': {}}, polyline={'shapeOptions': {}}),), layers=(TileLayer(attribution='Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size'], url='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'),), layout=Layout(height='400px'), options=['attribution_control', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=3)

In [11]:
# Get information from the map's drawing control.
dc.last_draw

{'geometry': None, 'type': 'Feature'}

To display an Earth Engine generated image on the interactive map, we can use ipyleaflet's TileLayer object. First we start by defining a function that can generate a tile layer URL from an Earth Engine image object.

In [12]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [13]:
# Style the image.
tile_url = GetTileLayerUrl(sample_image.visualize(min=0, max=3000, gamma=1.5, bands= ['red', 'green', 'blue']))
map1.add_layer(ipyleaflet.TileLayer(url=tile_url))

Lets redefine our sample, expanding the date range and getting rid of out geometry filters.

In [14]:
map2 = ipyleaflet.Map(zoom=3, layout={'height':'400px'})
map2

Map(center=[0.0, 0.0], layers=(TileLayer(attribution='Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size'], url='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'),), layout=Layout(height='400px'), options=['attribution_control', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=3)

In [15]:
def ReplaceOverlayLayers(map_object, ee_image_object):
    for lyr in map_object.layers[1:]:
        map_object.remove_layer(lyr)
    tile_url = GetTileLayerUrl(ee_image_object)
    map_object.add_layer(ipyleaflet.TileLayer(url=tile_url))
        
filtered = (
    l8sr.filterDate('2016-01-01', '2016-02-02')
         .median()
#         .mean()
#         .max()
#         .min()
#        .reduce(ee.Reducer.percentile([25])).rename(band_names_new)
        .visualize(min=0, max=3000, bands=['red', 'green', 'blue'])
)
ReplaceOverlayLayers(map2, filtered)

# Calculate a Spectral Index (NDVI)

In [16]:
IPython.display.Image(url='https://landsat.usgs.gov/sites/default/files/images/LandsatSpectralBands_20160801.jpg')

The normalized difference vegetation index ($NDVI$) is a band ratio that is related to the amount of green vegetation. 

\begin{equation*}
NDVI = \frac{NIR — RED}{NIR + RED} = \frac{Band5 — Band4}{Band5 + Band4}
\end{equation*}

where $NIR$ is the near infrared band and $RED$ is red band.

## Write a function to add an NDVI band to an image.

In [17]:
def AddBandNDVI(img):
    red = img.select('red')
    nir = img.select('nir')
    ndvi = (nir.subtract(red)).divide(nir.add(red)).rename('ndvi')
    return img.addBands(ndvi)

l8sr = l8sr.map(AddBandNDVI)

In [18]:
map3 = ipyleaflet.Map(
    zoom=4,
    layout={'height':'400px'}
)
map3

Map(center=[0.0, 0.0], layers=(TileLayer(attribution='Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size'], url='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'),), layout=Layout(height='400px'), options=['attribution_control', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=4)

In [19]:
map3.add_layer(
    ipyleaflet.TileLayer(
        url=GetTileLayerUrl(
            l8sr.filterDate('2014-01-01', '2014-10-01')
                    .max()
                    .visualize(bands=['ndvi'], min=-1, max=1, palette='grey,black,lime')
        )
    )
)

# Time Series

In [20]:
def GetDataFrame(coords):
    
    pnt = ee.Geometry.Point(coords)
    # Sample for a time series of values at the point.
    geom_values = l8sr.filterBounds(pnt).select('ndvi').getRegion(geometry=pnt, scale=30)
    geom_values_list = ee.List(geom_values).getInfo()
    # Convert to a Pandas DataFrame.
    header = geom_values_list[0]
    data = pd.DataFrame(geom_values_list[1:], columns=header)
    data['datetime'] = pd.to_datetime(data['time'], unit='ms', utc=True)
    data.set_index('time')
    data = data.sort_values('datetime')
    data = data[['datetime', 'ndvi']]
    return data

In [21]:
# Plot scales.
lc1_x = bqplot.DateScale(min=datetime.date(2013, 2, 1), max=datetime.date(2018, 1, 1))
lc2_y = bqplot.LinearScale()

# Plot type (mark).
lc2 = bqplot.Lines(
    x=[],
    y=[],
    scales={'x': lc1_x, 'y': lc2_y}, 
    display_legend=True,
)

# Plot axes.
x_ax1 = bqplot.Axis(label='Date', scale=lc1_x, num_ticks = 6, tick_format='%Y-%b')
x_ay2 = bqplot.Axis(label='NDVI', scale=lc2_y, orientation='vertical')

# Declare the plot interactions.
br_intsel = bqplot.interacts.BrushIntervalSelector(scale=lc1_x, marks=[lc2])

# Create a figure.
fig = bqplot.Figure(
    marks=[lc2],
    axes=[x_ax1, x_ay2],
    layout={'height':'300px', 'width':'100%'},
    interaction=br_intsel
)

# Create a map widget with a drawing control.
map5 = ipyleaflet.Map(zoom=3)
dc = ipyleaflet.DrawControl(polyline={}, polygon={})
map5.add_control(dc)

int_start_dp = ipywidgets.DatePicker(
    description='Start Date',
    disabled=True
)
int_end_dp = ipywidgets.DatePicker(
    description='End Date',
    disabled=True
)

# Create the event handlers for the map and plot.
def handle_draw(self, action, geo_json):
    # Get the selected coordinates from the map's drawing control.
    coords = geo_json['geometry']['coordinates']    
    new_df = GetDataFrame(coords)
    lc2.x = new_df['datetime']
    lc2.y = new_df['ndvi']
dc.on_draw(handle_draw)

def brush_selection_callback(change):
    (t1_start, t1_end) = change.new
    start_datetime = dateutil.parser.parse(t1_start)
    end_datetime = dateutil.parser.parse(t1_end)
    int_start_dp.value = start_datetime
    int_end_dp.value = end_datetime
    
    # Update the layer displayed on the map.
    filtered = (
        l8sr.filterDate(start_datetime.isoformat(), end_datetime.isoformat())
            .max()
            .visualize(bands=['ndvi'], min=0, max=1, palette='grey,green,lime')
    )
    ReplaceOverlayLayers(map5, filtered)    
br_intsel.observe(brush_selection_callback, names=['selected'])

# Display the widgets.
vbox = ipywidgets.VBox(
    [
        map5,
        ipywidgets.HBox([
            fig, ipywidgets.VBox([
                ipywidgets.HTML('Selected Dates:'),
                int_start_dp,
                int_end_dp
            ])
        ])
    ],
    align_self='stretch'
)

In [22]:
vbox

VBox(children=(Map(center=[0.0, 0.0], controls=(DrawControl(layer=FeatureGroup()),), layers=(TileLayer(attribution='Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size'], url='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'),), layout=Layout(align_self='stretch', height='400px'), options=['attribution_control', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=3), HBox(children=(Figure(axes=[Axis(label='Date', num_ticks=6, scale=DateScale(max=datetime.datetime(2018, 1, 1, 0, 0), min=datetime.datetime(2013, 2, 1, 0, 0)), tick_format='%Y-%b'), Axis(label='NDVI', orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, interaction=BrushIntervalSelector(marks=[Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], display_legend=True, interactions={'hover': 'tooltip'}, scales={'x': DateScale(max=datetime.datetime(2018, 1, 1, 0, 0), min=datetime.datetime(2013, 2, 1, 0, 0)), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9})], scale=DateScale(max=datetime.datetime(2018, 1, 1, 0, 0), min=datetime.datetime(2013, 2, 1, 0, 0))), layout=Layout(height='300px', width='100%'), marks=[Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], display_legend=True, interactions={'hover': 'tooltip'}, scales={'x': DateScale(max=datetime.datetime(2018, 1, 1, 0, 0), min=datetime.datetime(2013, 2, 1, 0, 0)), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9})], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0)), VBox(children=(HTML(value='Selected Dates:'), DatePicker(value=None, description='Start Date', disabled=True), DatePicker(value=None, description='End Date', disabled=True)))))))